In [1]:
import pandas as pd
import numpy as np
import os
import requests
import re

!pip install transformers -q
!pip install wandb -q

import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

!pip install SentencePiece -q

import torch.nn.functional as F
from transformers import  T5Tokenizer , T5ForConditionalGeneration
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

%pip install -q wandb
import wandb
wandb.login()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Note: you may need to restart the kernel to use updated packages.


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [13]:
df = pd.read_csv('/kaggle/input/data22/test.csv')

In [3]:
df

,id,Article,Heading,Summary
0,de5ea44ef434c0aa902814ae7b2a8767,Fear shakes Mexico border city after violence ...,Fear shakes Mexico border city after violence ...,The attacks took place in several neighborhood...
1,76d57cdfc8d87fa772c4b4be040bfa13,Indian-origin boy finds millions of years old ...,Indian-origin boy finds millions of years old ...,"Siddak Singh Jhamat, known as Sid, was using a..."
2,cf0653a693d70602f978c3ae589ef676,Representative ImageA 38-year-old Indian man h...,Indian charged with smuggling foreign national...,A 38-year-old Indian man has been charged with...
3,57436489483aec6a8e0a4f638dcf6250,Residents get tested during their stay at a te...,COVID: Shanghai extends lockdown till April 26...,"China on Apr 21 reported 2,119 locally transmi..."
4,4c78547b94fe84814a56e2c3342ef0c9,Israeli PM Benjamin Netanyahu flew to Saudi Ar...,Israeli PM Benjamin Netanyahu flew to Saudi Ar...,Israeli media reported Monday that Prime Minis...
...,...,...,...,...
4482,cc96456ee3d18e4ce8a2ee984e8efc46,FILE - Tanks storing treated radioactive water...,Japan approves plan to release millions of ton...,A massive earthquake and tsunami in 2011 destr...
4483,6049646fb8d132cc020c37b3136e3428,Villagers clear debris caused by an earthquake...,"16 killed, hundreds stranded on Indonesian mou...","Over 680 tourists from France, Thailand, the N..."
4484,0be5c6c3e574681ee40867a94a6c0986,"Seif al-Islam, the son and one-time heir appar...",Gadhafi’s son Seif al-Islam released after mor...,"Seif al-Islam, the son and one-time heir appar..."
4485,d58d4de72da6767081d70ce0f900b79f,Thai BoysMost members of the Thai youth footba...,Rescued Thai boys to be ordained in Buddhist c...,Most members of the Thai youth football team r...


In [4]:
wandb.init(project="English_t5_pred")

config = wandb.config         
config.TRAIN_BATCH_SIZE = 2  
config.VALID_BATCH_SIZE = 2    
config.TRAIN_EPOCHS = 8       
config.VAL_EPOCHS = 8 
config.LEARNING_RATE = 5e-5    
config.SEED = 42               
config.MAX_LEN = 512
config.SUMMARY_LEN = 75 


torch.manual_seed(config.SEED) 
np.random.seed(config.SEED)
torch.backends.cudnn.deterministic = True


tokenizer = T5Tokenizer.from_pretrained("t5-base")

wandb: Currently logged in as: ashutoshguptanitk (nitk_ai). Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:220: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes 

In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [8]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.text = self.data.text
        self.ctext = self.data.ctext

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        ctext = str(self.ctext[index])
        ctext = ' '.join(ctext.split())

        text = str(self.text[index])
        text = ' '.join(text.split())

        source = self.tokenizer.batch_encode_plus([ctext], max_length= self.source_len, pad_to_max_length=True,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([text], max_length= self.summ_len, pad_to_max_length=True,return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long),
            'source_mask': source_mask.to(dtype=torch.long),
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }


In [9]:
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask,
                max_length=75,
                num_beams=4,
                repetition_penalty=2.5,
                length_penalty=1.0,
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%100==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

In [14]:
df.drop(['id','Heading'], axis =1, inplace = True)
df.rename(columns = {'Article':'ctext', 'Summary':'text'}, inplace = True)
df.ctext = 'summarize: ' + df.ctext

In [15]:
df

,ctext,text
0,summarize: Fear shakes Mexico border city afte...,The attacks took place in several neighborhood...
1,summarize: Indian-origin boy finds millions of...,"Siddak Singh Jhamat, known as Sid, was using a..."
2,summarize: Representative ImageA 38-year-old I...,A 38-year-old Indian man has been charged with...
3,summarize: Residents get tested during their s...,"China on Apr 21 reported 2,119 locally transmi..."
4,summarize: Israeli PM Benjamin Netanyahu flew ...,Israeli media reported Monday that Prime Minis...
...,...,...
4482,summarize: FILE - Tanks storing treated radioa...,A massive earthquake and tsunami in 2011 destr...
4483,summarize: Villagers clear debris caused by an...,"Over 680 tourists from France, Thailand, the N..."
4484,"summarize: Seif al-Islam, the son and one-time...","Seif al-Islam, the son and one-time heir appar..."
4485,summarize: Thai BoysMost members of the Thai y...,Most members of the Thai youth football team r...


In [16]:
val_dataset = df

In [20]:
val_set = CustomDataset(val_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

# parameters fo dataloaders
train_params = {
    'batch_size': config.TRAIN_BATCH_SIZE,
    'shuffle': True,
    'num_workers': 0
    }

val_params = {
    'batch_size': config.VALID_BATCH_SIZE,
    'shuffle': False,
    'num_workers': 0
    }

# Create  Dataloaders 
val_loader = DataLoader(val_set, **val_params)


model = T5ForConditionalGeneration.from_pretrained("saliq7/T5_English")
model = model.to(device)


optimizer = torch.optim.Adam(params =  model.parameters(), lr=config.LEARNING_RATE)
wandb.watch(model, log="all")


#for epoch in range(config.VAL_EPOCHS):
predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
final_df.to_csv('predictions_english_t5.csv')
print('Output Files generated for review')

Completed 0
Completed 100
Completed 200
Completed 300
Completed 400
Completed 500
Completed 600
Completed 700
Completed 800
Completed 900
Completed 1000
Completed 1100
Completed 1200
Completed 1300
Completed 1400
Completed 1500
Completed 1600
Completed 1700
Completed 1800
Completed 1900
Completed 2000
Completed 2100
Completed 2200
Output Files generated for review


In [23]:
df = pd.read_csv('/kaggle/working/predictions_english_t5.csv')
df.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [37]:
df['Generated Text'][3]

'Symptomatic cases fell 26.7 per cent to 1,931, in the biggest one-day decline since the outbreak, Hong Kong-based South China Morning Post reported on Friday.'

In [36]:
df['Actual Text'][3]

'China on Apr 21 reported 2,119 locally transmitted confirmed COVID cases, of which 1,931 were reported in Shanghai, according to National Health Commission.'

In [34]:
df

,Generated Text,Actual Text
0,fear has invaded the Mexican border city of Re...,The attacks took place in several neighborhood...
1,"Siddak Singh Jhamat, known as Sid, was using a...","Siddak Singh Jhamat, known as Sid, was using a..."
2,28-year-old Indian man has been charged with c...,A 38-year-old Indian man has been charged with...
3,"Symptomatic cases fell 26.7 per cent to 1,931,...","China on Apr 21 reported 2,119 locally transmi..."
4,Israeli media reported Monday that Prime Minis...,Israeli media reported Monday that Prime Minis...
...,...,...
4482,plan was submitted by the Tokyo Electric Power...,A massive earthquake and tsunami in 2011 destr...
4483,magnitude 6.4 earthquake early Sunday killed 1...,"Over 680 tourists from France, Thailand, the N..."
4484,son and one-time heir apparent of late Libyan ...,"Seif al-Islam, the son and one-time heir appar..."
4485,Thai officials said the boys will start the pr...,Most members of the Thai youth football team r...


In [38]:
!pip install evaluate -q
!pip install rouge-score -q

In [39]:
import evaluate
rouge = evaluate.load('rouge')
predictions = [i for i in df['Generated Text']]
references = [i for i in df['Actual Text']]
results = rouge.compute(predictions=predictions, references=references)
print(results)

{'rouge1': 0.471234651307733, 'rouge2': 0.3583083940807778, 'rougeL': 0.42856081812945984, 'rougeLsum': 0.4282404579109943}


In [43]:
import nltk
from nltk.translate.bleu_score import corpus_bleu
score = corpus_bleu(references, predictions)
print(score)

0.6297854632360553


/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
